### Building NGC graphs

Construct Graph\
\
<img src="Group42.png" alt="MarineGEO circle logo" style="height: 200px; width:100px;"/>




# Learning a Data Generating Process


In [21]:
from ngclearn.generator.static.mog import MoG
import numpy as np
import tensorflow as tf
from construction_utils import SNodeBuilder, CableConnector
from ngclearn.engine.nodes.enode import ENode
from ngclearn.engine.ngc_graph import NGCGraph

In [22]:
mu1 = np.array([[-1.0,1.2]])
cov1 = np.array([[0.1,0.0],
                 [0.0,0.2]])
mu1 = tf.cast(mu1,dtype=tf.float32)
cov1 = tf.cast(cov1,dtype=tf.float32)

mu2 = np.array([[0.85,-1.3]])
cov2 = np.array([[0.2,0.0],
                 [0.0,0.12]])
mu2 = tf.cast(mu2,dtype=tf.float32)
cov2 = tf.cast(cov2,dtype=tf.float32)

mu3 = np.array([[0.92, 1.1]])
cov3 = np.array([[0.12,0.0],
                 [0.0,0.18]])
mu3 = tf.cast(mu3,dtype=tf.float32)
cov3 = tf.cast(cov3,dtype=tf.float32)


In [23]:
mu_list = [mu1, mu2, mu3]
sigma_list = [cov1, cov2, cov3]
process = MoG(means=mu_list, covar=sigma_list, seed=69)
x_dim = mu1.shape[1]

In [24]:
K = 40
leak = 0.0001
beta = 0.1
z2_dim = 2
z1_dim = 64

bd = SNodeBuilder()
cc = CableConnector()

z2 = bd                                   \
    .O1_set_numerals(beta=beta, leak=leak) \
    .O2_set_cats(integrate=True, use_dfx=True)              \
    .O0_build(name="z2", dim=z2_dim)                            #O
mu1 = bd.O1_set_numerals(zeta=0.0).O0_build("mu1", dim=z1_dim)  #V
e1 = ENode(name="e1", dim=z1_dim)                               #E
z1 = bd                                            \
    .O1_set_numerals(beta=beta, leak=leak)          \
    .O2_set_cats(integrate=True, use_dfx=True, act_fx=bd.ACT_FX.RELU) \
    .Op1_with_prior(pt=bd.PT.LAPLACE, lbd=0.0001)     \
    .O0_build("z1", dim=z1_dim)                                #^
mu0 = bd.O1_set_numerals(zeta=0.0).O0_build("mu0", dim=x_dim)  #0
e0 = ENode(name="e1", dim=x_dim)                               #E
z0 = bd                        \
    .O1_set_numerals(beta=beta) \
    .O2_set_cats(integrate=True, use_dfx=True)   \
    .O0_build(name="z0", dim=x_dim)                             #O

In [25]:
z2_mu1 = cc.O1_dense().Op1_with_update_rule(z2, e1).Op2_with_constraints().O0_connect(z2, mu1)
cc.O1_simple().O0_connect(mu1, e1, to_comp=cc.EComps.PMU)
cc.O1_simple().O0_connect(z1, e1, from_comp=cc.SComps.Z, to_comp=cc.EComps.PTARG)
cc.O1_mirror(z2_mu1).O0_connect(e1, z2, to_comp=cc.SComps.BU)
cc.O1_simple(-1.0).O0_connect(e1, z1)

z1_mu0 = cc.O1_dense().Op1_with_update_rule(z1, e0).Op2_with_constraints().O0_connect(z1, mu0)
cc.O1_simple().O0_connect(mu0, e0, to_comp=cc.EComps.PMU)
cc.O1_simple().O0_connect(z0, e0, from_comp=cc.SComps.Z, to_comp=cc.EComps.PTARG)
cc.O1_mirror(z1_mu0).O0_connect(e0, z1, to_comp=cc.SComps.BU)
cc.O1_simple(-1.0).O0_connect(e0, z0)

Connecting nodes z2-mu1 with -dense- cable
Connecting nodes mu1-e1 with -simple- cable
Connecting nodes z1-e1 with -simple- cable
Connecting nodes e1-z2 with -mirrored- cable
Connecting nodes e1-z1 with -simple- cable
Connecting nodes z1-mu0 with -dense- cable
Connecting nodes mu0-e1 with -simple- cable
Connecting nodes z0-e1 with -simple- cable
Connecting nodes e1-z1 with -mirrored- cable
Connecting nodes e1-z0 with -simple- cable


In [26]:
model = NGCGraph(K=K)
model.set_cycle(nodes=[z2, z1, z0])
model.set_cycle(nodes=[mu1, mu0])
model.set_cycle(nodes=[e1, e0])
model.apply_constraints()
info = model.compile(batch_size=1)

In [27]:
eta = 0.002
eta_v  = tf.Variable( eta ) # set up optimization process
opt = tf.keras.optimizers.legacy.Adam(eta_v)

In [3]:
from supercollider import Server, Synth

server = Server()

synth = Synth(server, "sine", { "freq" : 440.0, "gain" : -12.0 })
synth.set("freq", 220.0)



In [4]:
synth.set("freq", 440.0)

In [17]:
from pynput.mouse import Button, Controller

from supercollider import Server, Synth

server = Server()

synth1 = Synth(server, "sine", { "freq" : 440.0, "gain" : -12.0 })
synth2 = Synth(server, "sine", { "freq" : 440.0, "gain" : -12.0 })


mouse = Controller()
try:
    while(True):
        synth1.set("freq", mouse.position[0]/3.0)
        synth2.set("freq", mouse.position[1]/3.0)
except KeyboardInterrupt:
    synth1.free()
    synth2.free()



In [21]:
from construction_utils import SNodeBuilder, CableConnector
from ngclearn.engine.nodes.enode import ENode
from ngclearn.engine.ngc_graph import NGCGraph
x_dim = 2
z1_dim = 16
z2_dim = 3
bd = SNodeBuilder()
cc = CableConnector()

z2 = bd.O0_build("z2", dim=z2_dim)
mu1 = bd.O0_build("mu1", dim=z1_dim)
e1 = ENode("e1", dim=z1_dim)
z1 = bd.O0_build("z1", dim=z1_dim)
mu0 = bd.O0_build("mu0", dim=x_dim)
e0 = ENode("e0", dim=x_dim)
z0 = bd.O0_build("z0", dim=x_dim)

cc.O1_dense().O0_connect(z2, mu1)
cc.O1_dense().O0_connect(z1, mu0)


circuit = NGCGraph(K=1)
circuit.set_cycle([z2, z1, z0])
circuit.set_cycle([mu1, mu0])
circuit.set_cycle([e1, e0])
circuit.compile(batch_size=1)

import ngclearn.utils.experimental.viz_graph_utils as viz

viz.visualize_graph(circuit) # generate the graph visual of